# Updating basic communication between lib and Bernie firmware

In [1]:
import serial
import sys
import time
import re

In [2]:
import bernielib as bl

In [3]:
def listSerialPorts():
    """ 
    Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [4]:
listSerialPorts()

['COM5', 'COM6']

# Testing basic initialization

In [5]:
ber = bl.robot(cartesian_port_name='COM5', loadcell_port_name='COM6')

In [6]:
ber._readAll(ber.cartesian_port)

''

In [7]:
ber._readAll(ber.loadcell_port)

"Arnie's mobile gripper tool\r\nRev. 2.0, 3/21/2020\r\nPower servo up: P on\r\nPower servo down: P off\r\nRotate servo to 30 degrees: G0 30\r\nServo will actually operate after P on.\r\nTare all load cells: T\r\nTare left load cell: T L\r\nTare right load cell: T R\r\nGet left load cell reading: RL\r\nGet right load cell reading: RR\r\n"

In [8]:
ber._writeAndWait(ber.cartesian_port, 'version', '\r', '')

'Build version: edge-0565b13, Build date: Jun 19 2021 16:12:18, MCU: LPC1769, System Clock: 120MHz\r\n5 axis\n'

# Testing load cells communications

In [9]:
ber.writeAndWaitLoadCell('RR')

'-0.08\r\n'

In [10]:
ber.writeLoadCell('T')

In [11]:
ber.writeAndWaitLoadCell('RR')

'0.01\r\n'

In [12]:
ber.writeAndWaitLoadCell('RL')

'0.06\r\n'

# Cartesian movements

In [16]:
ber.writeAndWaitCartesian('G0 Y-10 F1000')
ber.writeAndWaitCartesian('M400')

'ok\n'

# Endstop status

In [9]:
# Nothing
ber.writeAndWaitCartesian('M119')

'X_min:0 Y_min:0 Z_min:0 A_min:0 pins- (X)P1.24:0 (Y)P1.26:0 (Z)P1.28:0 (A)P1.29:0 \nok\n'

In [10]:
# X
ber.writeAndWaitCartesian('M119')

'X_min:1 Y_min:0 Z_min:0 A_min:0 pins- (X)P1.24:1 (Y)P1.26:0 (Z)P1.28:0 (A)P1.29:0 \nok\n'

In [11]:
# Y
ber.writeAndWaitCartesian('M119')

'X_min:0 Y_min:1 Z_min:0 A_min:0 pins- (X)P1.24:0 (Y)P1.26:1 (Z)P1.28:0 (A)P1.29:0 \nok\n'

In [12]:
# Z
ber.writeAndWaitCartesian('M119')

'X_min:0 Y_min:0 Z_min:1 A_min:0 pins- (X)P1.24:0 (Y)P1.26:0 (Z)P1.28:1 (A)P1.29:0 \nok\n'

In [13]:
# A
ber.writeAndWaitCartesian('M119')

'X_min:0 Y_min:0 Z_min:0 A_min:1 pins- (X)P1.24:0 (Y)P1.26:0 (Z)P1.28:0 (A)P1.29:1 \nok\n'

# Homing

In [9]:
ber.writeAndWaitCartesian('$H')

# Homing

'ok\n'

In [10]:
ber.robotHome()

In [9]:
ber.robotHome(axis='X')

In [11]:
ber.home(part='pipette')

# Magnets

In [11]:
ber.rackPowerOn()
ber.rackMoveMagnetsAngle(5.2)
ber.rackPowerOff()

In [12]:
ber.rackPowerOn()
ber.rackMoveMagnetsAngle(11.2)
ber.rackPowerOff()

## Saving magnets settings for Bernie SN3

In [12]:
ber.setMagnetsAwayAngle(5.2)

In [13]:
ber.setMagnetsNearTubeAngle(11.2)

In [14]:
ber.getMagnetsAwayAngle(), ber.getMagnetsNearTubeAngle()

(5.2, 11.2)

In [21]:
ber.moveMagnetsAway()

In [22]:
ber.moveMagnetsTowardsTube()

## Homing the magnets

In [17]:
ber.home(part='magrack')

## Full homing

In [9]:
ber.home()

# Testing pipette servo

In [11]:
ber.pipetteServoPowerUp()
ber.movePipetteServoAngle(7.3)
ber.pipetteServoPowerDown()

In [12]:
ber.pipetteServoPowerUp()
ber.movePipetteServoAngle(2.5)
ber.pipetteServoPowerDown()

In [9]:
ber.setTipDropServoUpAngle(2.5)
ber.setTipDropServoDownAngle(7.3)

In [12]:
ber.pipetteServoDown()

In [13]:
ber.pipetteServoUp()

# Finding maximum speeds

In [20]:
ber.writeAndWaitCartesian('G0 Z150 F50000')

'ok\n'

In [21]:
ber.writeAndWaitCartesian('G0 Z10 F50000')

'ok\n'

In [73]:
ber.writeAndWaitCartesian('G0 X150 F100000')

'ok\n'

In [74]:
ber.writeAndWaitCartesian('G0 X10 F100000')

'ok\n'

In [75]:
ber.writeAndWaitCartesian('G0 Y200 F50000')

'ok\n'

In [76]:
ber.writeAndWaitCartesian('G0 Y10 F50000')

'ok\n'

In [89]:
ber.writeAndWaitCartesian('G0 A35 F3000')

'ok\n'

In [90]:
ber.writeAndWaitCartesian('G0 A3 F3000')

'ok\n'

## Setting up speeds

In [9]:
ber.setSpeedXY(50000)
ber.setSpeedZ(30000)
ber.setSpeedPipette(2500)

In [10]:
ber.getSpeedXY(), ber.getSpeedZ(), ber.getSpeedPipette()

(50000, 30000, 3000)

In [11]:
ber.getSpeed('X'), ber.getSpeed('Y'), ber.getSpeed('Z'), ber.getSpeed('A'), ber.getSpeed('pipette'), ber.getSpeed('plunger')

(50000, 50000, 30000, 3000, 3000, 3000)

# Movement commands

In [14]:
ber.moveAxis('X', 10)

In [15]:
ber.moveAxis('Y', 20)

In [17]:
ber.moveAxis('Z', 1)

In [22]:
ber.moveAxis('A', 3)

## Get position

In [17]:
ber.getPosition()

(10.0, 20.0, 1.0, 3.0)

# Turn off motor

In [26]:
ber.writeAndWaitCartesian('M18 A0')

'ok\n'

In [24]:
ber.writeAndWaitCartesian('M17')

'ok\n'

# Pipette functions

In [10]:
ber.pipetteHome()

In [12]:
ber.pipetteMove(3)

In [13]:
ber.pipetteServoDown()

In [14]:
ber.pipetteServoUp()

# Approach until touch

In [20]:
ber.move(x=120, y=90, z=150, z_first=False)

In [22]:
ber.moveDownUntilPress(0.2, 1000)

152.2

In [23]:
ber.getPosition()

(120.0, 90.0, 152.2, 3.0)

In [24]:
ber.move(x=120, y=90, z=10, z_first=False)

In [25]:
ber.writeAndWaitCartesian('M18 A0')

'ok\n'

In [26]:
ber.powerStepperOff('A')

In [27]:
ber.powerStepperOff('X')

In [28]:
ber.powerStepperOff('Y')

In [29]:
ber.powerStepperOff('Z')